<a href="https://colab.research.google.com/github/donghuna/PromptGenerate/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install langchain
!pip install tiktoken
!pip install langchain-openai
!pip install faiss-cpu
!pip install langchain-community
!pip install sentence_transformers
!pip install peft
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 50.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 31.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.1 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 47.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49

In [6]:
import os
from pathlib import Path
import subprocess
import shutil
from langchain.text_splitter import TokenTextSplitter

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_openai import ChatOpenAI
from langchain.retrievers.document_compressors import LLMChainExtractor
from langchain.retrievers import ContextualCompressionRetriever
from langchain.tools.retriever import create_retriever_tool
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

import pandas as pd
import openai
import faiss
import numpy as np

from datetime import datetime
import transformers
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from sentence_transformers import SentenceTransformer
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model, AutoPeftModel, PeftModel
from datasets import load_dataset

import pickle

In [7]:
# 모델 경로
model_path = "sentence-transformers/paraphrase-MiniLM-L6-v2"

# GPU 확인
device = "cuda" if torch.cuda.is_available() else "cpu"

# 모델 로드
model = SentenceTransformer(model_path, device=device)

model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.73k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [8]:
# Load csv fileㅇㅇ
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
import pandas as pd

# CSV 파일 경로
file_path = '/content/drive/MyDrive/generated_similarQuestion.csv'

# CSV 파일 읽어오기
df = pd.read_csv(file_path)

# 데이터프레임 확인
print(df.columns)
print(df.head())


Index(['question_id', 'question', 'snippet', 'similarQuestion'], dtype='object')
   question_id                                           question  \
0     18601725  Using Regex (or simmilar) to change song name ...   
1     21889025                  Product of two beta distributions   
2     26661808           how to grab from JSON in selenium python   
3      7523511  Unable to use wx.NotificationMessage properly ...   
4     34996225          Django: Add additional properties to User   

                                             snippet  \
0  set ASTID to AppleScript's text item delimiter...   
1  set.seed(1) # for reproducability\n\nn <- 1000...   
2  from bs4 import BeautifulSoup\nimport json\n\n...   
3  import wx, sys\n\ntry:\n    import win32gui #,...   
4  if request.method == 'POST':\n    new_user = U...   

                                     similarQuestion  
0  How to modify song names in iTunes by replacin...  
1  How can we generate and visualize the distribu...  
2 

In [10]:
document_ids = []
embeddings = []

for i, question in enumerate(df['question'].tolist()):
    embedding = model.encode(question)
    embeddings.append(embedding)
    document_ids.append(i)

with open('embeddings_ids-sentence.pkl', 'wb') as f:
    pickle.dump((document_ids, embeddings), f)

def normalize(vectors):
    norms = np.linalg.norm(vectors, axis=1, keepdims=True)
    return vectors / norms

# 임베딩을 Numpy 배열로 변환
embedding_matrix = np.array(embeddings).astype('float32')

# 정규화
embedding_matrix_normalized = normalize(embedding_matrix)

# FAISS 인덱스 생성 및 벡터 추가
index = faiss.IndexFlatIP(embedding_matrix_normalized.shape[1])
index.add(embedding_matrix_normalized)

# faiss 인덱스 저장
faiss.write_index(index, 'question_faiss_index-sentence.faiss')

In [11]:
print(f"Number of questions: {len(df)}")
print(f"Number of embeddings: {len(embeddings)}")

Number of questions: 100
Number of embeddings: 100


In [12]:
## FAISS 인덱스 로드

index = faiss.read_index('question_faiss_index-sentence.faiss')

with open('embeddings_ids-sentence.pkl', 'rb') as f:
    document_ids, embeddings = pickle.load(f)

In [19]:
## 데이터 로드

data_path = "/content/drive/MyDrive/generated_similarQuestion.csv"
similarQuestion = load_dataset("csv", data_files=data_path)
print(similarQuestion)

DatasetDict({
    train: Dataset({
        features: ['question_id', 'question', 'snippet', 'similarQuestion'],
        num_rows: 100
    })
})


In [21]:
# 유사도 검색 함수
def search_similar_questions(query_sentence, k=10):
    query_embedding = model.encode([query_sentence])
    query_embedding_normalized = normalize(query_embedding)

    # FAISS 인덱스에서 가장 유사한 벡터 k개 검색
    distances, indices = index.search(np.array([query_embedding_normalized[0]], dtype='float32'), k)

    # 검색된 인덱스와 유사도를 반환
    similar_questions = []
    for i in range(k):
        question = df['question'][indices[0][i]]
        snippet = df['snippet'][indices[0][i]]
        # plan = df['plan'][indices[0][i]]

        print(f"문서 id: {indices[0][i]}, 유사도: {distances[0][i]}, 질문: {question}")
        similar_questions.append((question, snippet))

    return similar_questions

In [25]:
## FAISS 인덱스에서 유사한 벡터 검색
idx = 27

query_sentence = similarQuestion['train']['similarQuestion'][idx]
original_question = similarQuestion['train']['question'][idx]
print(f"idx : {idx}, Query: {query_sentence}")

similar_questions = search_similar_questions(query_sentence, k=5)

idx : 27, Query: How to handle unknown skip error when reading a file in python
문서 id: 27, 유사도: 0.8734531998634338, 질문: Unknown skip error file read in python
문서 id: 81, 유사도: 0.44316530227661133, 질문: Why won't my python code print anything?
문서 id: 76, 유사도: 0.40594327449798584, 질문: Checking text files for certain strings in python
문서 id: 98, 유사도: 0.34318816661834717, 질문: Python equivallent of Perl match and capture within if block
문서 id: 93, 유사도: 0.3246307969093323, 질문: Match characters in a line and print the line before python


In [46]:
from google.colab import userdata
from ftplib import FTP

ftp_server = "121.136.96.223"
ftp_port = 21
ftp_username = "donghuna_ftp"
ftp_password = userdata.get('FTP_PASSWD')
folder_path = "database/PromptGenerate/"

ftp = FTP()
ftp.connect(ftp_server, ftp_port)
ftp.login(ftp_username, ftp_password)
ftp.set_pasv(True)
# ftp.cwd(folder_path)

In [48]:
# ftp 경로에 파일 저장
import io

csv_buffer = io.StringIO()
df.to_csv(csv_buffer, index=True)

csv_buffer.seek(0) # buffer의 시작으로 이동
ftp.storbinary(f"STOR {folder_path}test.csv", io.BytesIO(csv_buffer.getvalue().encode()))

'226 Transfer complete.'